In [1]:
import psi4
import numpy as np

In [2]:
# Initial setup
psi4.set_memory('2 GB')
psi4.set_num_threads(2)

file_prefix = 'methane_HF-DZ'

ch4 = psi4.geometry("""
symmetry c1
0 1
   C       -0.85972        2.41258        0.00000
   H        0.21028        2.41258        0.00000
   H       -1.21638        2.69390       -0.96879
   H       -1.21639        3.11091        0.72802
   H       -1.21639        1.43293        0.24076
""")

# Geometry optimization
psi4.set_output_file(file_prefix + '_geomopt.dat', False)
psi4.set_options({'g_convergence': 'gau_tight'})
psi4.optimize('scf/cc-pVDZ', molecule=ch4)

Optimizer: Optimization complete!


-40.19871710397676

In [3]:
# Run vibrational frequency analysis
psi4.set_output_file(file_prefix + '_vibfreq.dat', False)
scf_energy, scf_wfn = psi4.frequency('scf/cc-pVDZ', molecule=ch4, return_wfn=True, dertype='gradient')

# Save "raw" frequencies into a variable
Freq = scf_wfn.frequency_analysis
print(Freq) # this command is just to get you started!

 19 displacements needed.
 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
{'q': QCAspect(lbl='normal mode', units='a0 u^1/2', data=array([[-5.29948027e-02, -8.03029431e-03,  7.18331036e-02,
        -3.18965723e-01,  4.06266981e-01,  6.88266481e-01,
        -2.75475305e-01,  5.59329508e-02, -2.80607078e-01,
         2.14495142e-07,  4.39530803e-07,  2.78744869e-05,
         1.16933058e-02,  2.18711628e-02, -3.05112727e-01],
       [ 4.21553077e-01,  1.15918284e-01, -4.20172909e-01,
         5.40576178e-01,  6.31138006e-02,  2.90930225e-01,
         1.59473899e-01, -2.93395184e-01, -2.15039021e-01,
        -3.71292818e-06,  2.80484902e-06, -2.14021671e-07,
        -2.55011457e-01,  1.69332381e-01,  2.36457221e-03],
       [-3.23378456e-01,  5.80103318e-02,  5.34774299e-01,
         5.45174768e-01, -8.62365187e-02,  2.23519644e-01,
         2.37563416e-01,  2.61812003e-01, -1.81032323e-01,
         4.39852244e-06,  3.15681901e-06, -6.35491734e-06,
        -1.68945086e-01, -2.54082255e-01

In [13]:
# Eliminate imaginary parts of frequencies,
# round the frequencies (to the nearest whole number),
# and extract only the *non-zero* frequencies

nonZero = np.round(np.real(Freq["omega"][2]))[6:]
print(nonZero)

[1434. 1434. 1434. 1648. 1648. 3165. 3286. 3286. 3286.]


In [26]:
# Determine the unique non-zero frequencies and 
# the number of times each such frequency occurs;
# store these in a NumPy array in the format: 
# {frequency, count} (i.e, one line per freq.)
freqCount = np.array(np.unique(nonZero, return_counts=True))
freqCount = freqCount.T    

[[1.434e+03 3.000e+00]
 [1.648e+03 2.000e+00]
 [3.165e+03 1.000e+00]
 [3.286e+03 3.000e+00]]


In [ ]:
## Save the NumPy array with frequency and count data
# to a text file

np.savetxt('freqCount.txt', freqCount, fmt='%d %d', header='Freq Degen')